In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options


from bs4 import BeautifulSoup
import time
import pandas as pd
from pandas import DataFrame as DF
from IPython.display import display

from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

import smtplib
from email.mime.text import MIMEText


In [4]:
def get_page(browser,page=''):
    browser.get(page)
    time.sleep(1)
    
def click_button(browser,xpath=''):
    dummy=browser.find_element_by_xpath(xpath)
    dummy.click()
    time.sleep(1)
    
def send_keys(browser,xpath='',value=''):
    dummy=browser.find_element_by_xpath(xpath)
    dummy.send_keys(value)
    time.sleep(1)
    

In [8]:
db_dir='/media/disk3/feynman52/See26/crawler-ryh'
db_name='urls.db'
db_url = 'sqlite:///'+db_dir+'/'+db_name
db_url

'sqlite:////media/disk3/feynman52/See26/crawler-ryh/urls.db'

In [4]:
engine = create_engine(db_url, echo = False)
Base = declarative_base()

class Urls(Base):
    __tablename__ = 'urls'
    url_id = Column(Integer, primary_key=True)
    url_title = Column(String)
    url_link = Column(String)
    url_user = Column(String)
    url_hot = Column(String)
    
Base.metadata.create_all(engine)

# Urls.__table__.drop(engine)



In [5]:
def add_to_table(items):
    session.add_all(items)
    session.commit()
    
def get_session(engine):
    Session = sessionmaker(bind = engine)
    session = Session()
    return session

def get_old_urls(session):
    old_url_objects=session.query(Urls).order_by(Urls.url_id.desc()).limit(1000).all()
    return old_url_objects

session=get_session(engine)
# a=session.query(Urls).order_by(Urls.url_id.desc()).limit(3).all()
# b=session.query(Urls).order_by(Urls.url_id.desc()).limit(10).all()
# c=[x for x in b if x not in a]
# len(c)
# [x.url_title for x in c]

NameError: name 'engine' is not defined

In [6]:
def sql_to_df(query=''):
    engine = create_engine(db_url)
    df = pd.read_sql_query(query, engine)
    return df

# query='''
# select * from urls
# order by id 
# asc limit 2
# '''
# df=sql_to_df(query=query)
# df

In [7]:
def get_browser():
    options = Options()
    options.headless = True
    _browser_profile = webdriver.FirefoxProfile()
    _browser_profile.set_preference("dom.webnotifications.enabled", False)
    executable_path='/media/disk3/feynman52/dummy/SQLAlchemy/591/geckodriver'
    browser = webdriver.Firefox(executable_path=executable_path,
                                options=options,
                                firefox_profile=_browser_profile)
    return browser

# get urls

In [8]:

# browser=get_browser()
# get_page(browser,page='https://www.ptt.cc/bbs/Gossiping/index.html')
# click_button(browser,xpath='/html/body/div[2]/form/div[1]/button')


In [9]:
# get_page(browser,page='https://www.ptt.cc/bbs/Gossiping/index39067.html')
# pageSource = browser.page_source  
# current_page = BeautifulSoup(pageSource,features="lxml")  

In [10]:
# rows=current_page.find_all('div',{'class':'r-ent'})


In [11]:
# row=rows[11]

In [12]:
# url_user=row.find('div',{'class':'author'}).text.replace('\n','')
# url_user

In [13]:
# url_title=row.find('div',{'class':'title'}).text.replace('\n','')
# url_title

In [14]:
# url_hot=row.find('div',{'class':'nrec'}).text.replace('\n','')
# url_hot

In [15]:
# url_link='https://www.ptt.cc'+row.find('a')['href']
# url_link

In [16]:
def get_url_object(row):
    try:
        url_user=row.find('div',{'class':'author'}).text.replace('\n','')
        url_title=row.find('div',{'class':'title'}).text.replace('\n','')
        url_hot=row.find('div',{'class':'nrec'}).text.replace('\n','')
        url_link='https://www.ptt.cc'+row.find('a')['href']
        url_object=Urls(
            url_user=url_user,
            url_title=url_title,
            url_hot=url_hot,
            url_link=url_link,)
        return url_object
    except:
        return

In [17]:
# rows=[get_url_object(row) for row in rows if get_url_object(row)!=None]
# len(rows)

In [18]:
# add_to_table(rows)

In [19]:
# browser=get_browser()
# get_page(browser,page='https://www.ptt.cc/bbs/Gossiping/index.html')
# click_button(browser,xpath='/html/body/div[2]/form/div[1]/button')

# session=get_session(engine)


In [20]:
# # from old to new

# current=5
# while True:
#     get_page(browser,page='https://www.ptt.cc/bbs/Gossiping/index%s.html'%(current))
#     pageSource = browser.page_source  
#     current_page = BeautifulSoup(pageSource,features="lxml")  
    
#     rows=current_page.find_all('div',{'class':'r-ent'})
#     new_urls=[get_url_object(row) for row in rows if get_url_object(row)!=None]
    
#     old_urls=get_old_urls(session)
#     old_urls=[x.url_link for x in old_urls]
    
#     new_urls_after_remove=[x for x in new_urls if x.url_link not in old_urls]
    
#     print('---------')
#     print('current',current)
#     print('new_urls_after_remove',len(new_urls_after_remove))
    
#     add_to_table(new_urls_after_remove)
    
#     if current==9: break
    
#     current+=1
    

In [34]:
# def delete_from_table(session,item):
#     session.delete(item)
#     session.commit()
    
# xs = session.query(Urls).filter(Urls.url_id>782445)
# for x in xs:
#     print(x.url_id)
#     delete_from_table(session,x)


In [9]:
pd.set_option('display.max_colwidth', 60)

query='''
select * from urls
'''
df=sql_to_df(query=query)
df

,url_id,url_title,url_link,url_user,url_hot
0,1,第一耶！,https://www.ptt.cc/bbs/Gossiping/M.1119222611.A.7A9.html,gogoegg,74
1,2,Re: (問題)華航空難留言,https://www.ptt.cc/bbs/Gossiping/M.1119222660.A.94E.html,Muroi,爆
2,3,Re: 有沒有明天會更好的八卦,https://www.ptt.cc/bbs/Gossiping/M.1119233779.A.191.html,JCC,爆
3,4,Re: 請問還有濁水溪公設的八卦嗎,https://www.ptt.cc/bbs/Gossiping/M.1119240642.A.EB1.html,goddora,爆
4,5,Re: (問題)華航空難留言,https://www.ptt.cc/bbs/Gossiping/M.1119243418.A.790.html,MCI,爆
5,6,[八卦],https://www.ptt.cc/bbs/Gossiping/M.1119245434.A.830.html,killerangel,爆
6,7,Re: [問題]職棒似乎又出現問題了...,https://www.ptt.cc/bbs/Gossiping/M.1119247736.A.35B.html,three3,40
7,8,Re: [政商] 請問桌伯元的八卦?,https://www.ptt.cc/bbs/Gossiping/M.1119250669.A.EA3.html,J1,25
8,9,Re: [新聞] 張震月的八卦有嗎？,https://www.ptt.cc/bbs/Gossiping/M.1119257927.A.60D.html,vancie,57
9,10,Re: [政商] 請問桌伯元的八卦?,https://www.ptt.cc/bbs/Gossiping/M.1119258686.A.6FC.html,J1,68
